In [1]:
import pytesseract
import cv2
from PIL import Image 
import re
import pandas as pd

In [2]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [3]:
formats = pd.read_excel('invoice_formats.xlsx')
final = pd.read_excel('Extracted_Data.xlsx')

In [479]:
Ima = "Makro Invoices/Makro Invoices/Converted/03.07.20_Extintores Valle Union Sl_3243,8_page1.png"

In [482]:
%%time
dat = {
    "Vat Number":'',
       "Total":0,
       "Date":'',
      "Invoice_Number":''}
img  = cv2.imread(Ima, cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
vat = []
date = []
print(type(text_list))
for idx, a in enumerate(text_list):
    if(a.strip() != ""):


        if (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            print(b)
        elif (re.search("[0-9]{8}-[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}-[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
        elif (re.search("[0-9]{8}[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}[A-Z]{1}",a)
            dat["Vat Number"] = b
            vat.append(b)
    
        elif (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)

        elif (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            
        elif (re.search("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            
        elif (re.search("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)
            dat["Vat Number"] = b[0]
        elif (re.search("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]

        if len(vat)>1:
            dat["Vat Number"] = vat[0]
        else:
            pass
         #Date section           
        if (re.search("\d{2}/\d{2}/\d{4}",a)):
            b  = re.findall("\d{2}/\d{2}/\d{4}", a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("\d{2}-\d{2}-\d{4}",a)):
            b = re.findall("\d{2}-\d{2}-\d{4}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("\d{2}/\d{2}/\d{2}",a)):
            b = re.findall("\d{2}/\d{2}/\d{2}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("\d{1}/\d{2}/\d{2}",a)):
            b = re.findall("\d{1}/\d{2}/\d{2}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        
        elif (re.search("[0-9]{2}.[0-9]{2}.[0-9]{4}",a)):
            b = re.findall("[0-9]{2}.[0-9]{2}.[0-9]{4}",a)
            dat["Date"] = b[0]
            date.append(b[0])
    
        if len(date)>1:
            dat["Date"] = date[0]

        #invoice number section
        if ("Factura" in a or "ALBARAN" in a or "Factura: " in a):

            print(a)
            b = a.split(" ")
            if b[0] == "ALBARAN":
                dat["Invoice_Number"] = b[1]
            print(b)
            if len(b)>3:
                print(b)
                if len(b)>4 and b[2] !='—' and len(b[1])!=25 and len(b[1])>5:
                    dat["Invoice_Number"] = b[1]+b[2]
                    print(b)
                elif len(b)>=5 and len(b[1])==25:
                    dat["Invoice_Number"] = b[1]
                elif len(b)>6 and len(b[3])>15:
                    dat["Invoice_Number"] = b[1]+ b[2]+ b[3]
                elif len(b)>6 and b[1]== "0/0":
                    dat["Invoice_Number"] = b[1] + b[2]
                    
        if ("Tarjeta" in a or "Credito" in a or  "pagina" in a or "Total:" in a  or "TOTAL" in a or  "Total factura" in a or  "Total Factura" in a or "TOTAL FACTURA" in a or "Total CON IGIC" in a  or "IMPORTE FACTURA EU" in a or "IMPORTE FACTURA" in a or "EUROS" in a):
            print("Case Total")
            b = a.split(" ")
            print(b)
            if len(b[-1])==2:
                
                dat["Total"] = b[-2]+b[-1]
                print(b[-2]+b[-1])
                
            if b[-1]=='€':
                dat["Total"] = b[-2]
            if b[-1] == "|":
                dat["Total"] = b[-2]
            if b[-1] == "EUR":
                dat["Total"] = b[-2]
            else:
                dat["Total"] = b[-1]
                print(b[-1])
                
if dat["Total"] == 0 or len(dat['Total'])==1 or dat["Total"]=="SUB TOTAL"  or dat["Total"] == 'factura' or dat["Total"]=="FACTURA" or dat["Total"]== "EUROS" or dat["Total"] == "©" or dat["Total"]=="TOTAL":
        tot = []
        im = Image.open(Ima) 
        # Size of the image in pixels (size of orginal image) 
        # (This is not mandatory) 
        width, height = im.size 
        print(im.size)
        # Setting the points for cropped image 
        left = 605
        top = 600
        right = 1675
        bottom = 1590
        # Cropped image of above dimension 
        # (It will not change orginal image) 
        im1 = im.crop((left, top, right, bottom)) 
         # Shows the image in image viewer 
        im1.save("Makro Invoices/Makro Invoices/New/pp2.jpg")
        img  = cv2.imread('Makro Invoices/Makro Invoices/New/pp2.jpg', cv2.COLOR_RGB2GRAY)
        tex=(pytesseract.image_to_string(img,config='outputbase digits'))
        text_list = tex.split("\n")
        text_list = text_list
        print(text_list)
        for i in text_list:
            if len(i)>=4 and len(i)<=7:
                print(i)
                tot.append(i)
                dat["Total"] = max(tot)
            
            
        

<class 'list'>
Case Total
['i', 'NETO', 'DTO', 'PP.', 'B.IMPONIBLE', 'IVA', 'IMP.', 'L.V.A.', '%', 'REC.', 'IMP.', 'REC', 'TOTAL']
TOTAL
(1191, 1684)
['2500 224.00', '10.00', '', '60.00 2.100.00', '', '.', '', '3.243.80', '', '.-.', '.-.', '']
10.00
Wall time: 6.11 s


In [483]:
dat

{'Vat Number': 'B38574281',
 'Total': '10.00',
 'Date': '03/07/2020',
 'Invoice_Number': ''}

In [484]:
data  =  pd.DataFrame(dat,index=[0])
data['Vat Number']= data['Vat Number'].str.replace(' ', '')
print(data)

  Vat Number  Total        Date Invoice_Number
0  B38574281  10.00  03/07/2020               


In [485]:
data['Vat Number']= data['Vat Number'].str.replace('-', '')
data['Vat Number']= data['Vat Number'].str.replace(',', '')
data['Vat Number']= data['Vat Number'].str.replace('.', '')
data

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


,Vat Number,Total,Date,Invoice_Number
0,B38574281,10.00,03/07/2020,


In [486]:
formats['last']= (formats['nif'].isin(data['Vat Number']))

In [487]:
df2 = formats.query("last == True")

In [488]:
df2

,formato,nombre,nif,last
81,[A-Z]{1}[0-9]{1}-[0-9]{1}-[0-9]{5},L M Seguridad Sl,B38574281,True
338,[A-Z]{1}[0-9]{1}-[0-9]{1}-[0-9]{5},L M Seguridad Sl,B38574281,True
652,[A-Z]{1}[0-9]{1}-[0-9]{1}-[0-9]{5},L M Seguridad Sl,B38574281,True
1086,[A-Z]{1}[0-9]{1}-[0-9]{1}-[0-9]{5},L M Seguridad Sl,B38574281,True
2455,[A-Z]{1}[0-9]{1}-[0-9]{1}-[0-9]{5},L M Seguridad Sl,B38574281,True
3441,[A-Z]{1}[0-9]{1}-[0-9]{1}-[0-9]{5},L M Seguridad Sl,B38574281,True
4445,[A-Z]{1}[0-9]{1}-[0-9]{1}-[0-9]{5},L M Seguridad Sl,B38574281,True
4728,[A-Z]{1}[0-9]{1}-[0-9]{1}-[0-9]{5},L M Seguridad Sl,B38574281,True
5397,[A-Z]{1}[0-9]{1}-[0-9]{1}-[0-9]{5},L M Seguridad Sl,B38574281,True
5398,[A-Z]{1}[0-9]{1}-[0-9]{1}-[0-9]{5},L M Seguridad Sl,B38574281,True


In [489]:
inv = 0
for i in df2['formato']:
    inv = i
pat = 'F1-1-01353'
factura = re.compile(inv)
mo = factura.search(text)
dat["Invoice_Number"]=mo.group()

In [490]:
dat

{'Vat Number': 'B38574281',
 'Total': '10.00',
 'Date': '03/07/2020',
 'Invoice_Number': 'F1-1-01353'}

In [491]:
text

'Valleunion\nTelf.; *+34 968 893233 - Fax; ++34 968 893833 - valleunionextintoresy@zmail com\nFACTURA: F1-1-01353\nEXTINTORES CARLOS\n\nFecha: 03/07/2020 L.M.SEGURIDAD, S.L.\n\nNIF/VAT: B38574281 POLG.LOS MAJUELOS C/TIJARAFE N-3 BAJO\n\nCliente: 4300000470 38108 LA LAGUNA-STA.C.TENERIFE\n\nAgente: 0001 CANARIAS\n\nPag. | del\ni Cédigo Descripcién LOTE Cantidad Precio DTO Importe\n& 04625 RACORD MANGUERA 25 MM 6,86 171,50\n2= 04134 RACOR MANGUERA 45 ALUM. 6,30 157,50\nHi 04397 RACOR R/ INT 25 MM 6,30 220,50\n42 04396 RACORD R/EXT. 25 MM. 6,30 157,50\n04237 RACORD R/INT 45 MM 8,96 224,00\n\ni 04322 VALVULA LATON C/RACORD 25 MM 21,28 212,80\ni i SIN RACORD\n$$ 02501 EXTINTOR 5 KG. CO2 "VALLE-UNION" 60,00 35,00 2.100,00\nit\n#8 BULTO: I PALET\nA PESO: 950 KG\niF\n38\nHy\ni\n33 :\nHe\nHe\ne\n£\ni NETO DTO PP. B.IMPONIBLE IVA IMP. L.V.A. % REC. IMP. REC TOTAL\n4\n; 3.243,80 3.243,80 3.243,80 €|\na\nForma de Pago: Banco.:\n\nREPOSICION PARAGE 60/90 DIAS\nCédigo IBAN\n\ni. EES ssocuon .\n, — V

In [492]:
import boto3
import io

In [493]:
im = Image.open(Ima)

In [494]:
buffered = io.BytesIO()
im.save(buffered, format='PNG')
client = boto3.client('textract')
response = client.analyze_document(
    Document={'Bytes': buffered.getvalue()},
    FeatureTypes=['TABLES']
)

In [495]:
def map_blocks(blocks, block_type):
    return {
        block['Id']: block
        for block in blocks
        if block['BlockType'] == block_type
    }

blocks = response['Blocks']
tables = map_blocks(blocks, 'TABLE')
cells = map_blocks(blocks, 'CELL')
words = map_blocks(blocks, 'WORD')
selections = map_blocks(blocks, 'SELECTION_ELEMENT')

def get_children_ids(block):
    for rels in block.get('Relationships', []):
        if rels['Type'] == 'CHILD':
            yield from rels['Ids']
            
dataframes = []

for table in tables.values():

    # Determine all the cells that belong to this table
    table_cells = [cells[cell_id] for cell_id in get_children_ids(table)]

    # Determine the table's number of rows and columns
    n_rows = max(cell['RowIndex'] for cell in table_cells)
    n_cols = max(cell['ColumnIndex'] for cell in table_cells)
    content = [[None for _ in range(n_cols)] for _ in range(n_rows)]

    # Fill in each cell
    for cell in table_cells:
        cell_contents = [
            words[child_id]['Text']
            if child_id in words
            else selections[child_id]['SelectionStatus']
            for child_id in get_children_ids(cell)
        ]
        i = cell['RowIndex'] - 1
        j = cell['ColumnIndex'] - 1
        content[i][j] = ' '.join(cell_contents)

    # We assume that the first row corresponds to the column names
    dataframe = pd.DataFrame(content[1:], columns=content[0])
    dataframes.append(dataframe)

In [496]:
if len(dataframes)>1:
    if  "Concepto" in dataframes[1] or "MM Num. articulo" in dataframes[1] or "PRECIO" in dataframes[1] or "Precio" in dataframes[1] or "PRECIO/UD" in dataframes[1] or "PRECIO UD." in dataframes[1] or "Num. artículo" in dataframes[1] or "Código" in dataframes[1] or "FECHA" in dataframes[1] or  "ARTICULO" in dataframes[1] or  "PRECIO KG" in dataframes[1] or "CODIGO" in dataframes[1] or "Cod." in dataframes[1] or "ARTÍCULO" in dataframes[1] or "Tipo" in dataframes[1] or "PRECIO UD"  in dataframes[1]:
        df_t = pd.DataFrame(dataframes[1])
        df_t.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
        print("Yes 1")

if "Concepto" in dataframes[0] or "MM Num. articulo" in dataframes[0] or "PRECIO" in dataframes[0] or "Precio" in dataframes[0] or  "PRECIO/UD" in dataframes[0] or "Código" in dataframes[0] or "PRECIO UD." in dataframes[0]  or "FECHA" in dataframes[0] or "ARTICULO" in dataframes[0] or "PRECIO KG" in dataframes[0] or "CODIGO" in dataframes[0] or "Cod." in dataframes[0] or "ARTÍCULO" in dataframes[0] or "Tipo" in dataframes[0] or "PRECIO UD"  in dataframes[0]:
        df_t = pd.DataFrame(dataframes[0])
        df_t.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
        print("Yes 2")
if len(dataframes)>2:
    if "Concepto" in dataframes[2] or "MM Num. articulo" in dataframes[2] or "PRECIO" in dataframes[2] or "Precio" in dataframes[2] or  "PRECIO/UD" in dataframes[2] or "Código" in dataframes[2] or "PRECIO UD" in dataframes[2] or "PRECIO UD." in dataframes[2]  or "FECHA" in dataframes[2] or "ARTICULO" in dataframes[2] or "PRECIO KG" in dataframes[2] or "CODIGO" in dataframes[2] or "Cod." in dataframes[2] or "ARTÍCULO" in dataframes[2] or "Tipo" in dataframes[2]:
        df_t = pd.DataFrame(dataframes[2])
        df_t.to_excel('Makro Invoices/Makro Invoices/Saved_Datad.xlsx')
        print("Yes 3")


Yes 2


In [497]:
df_t

,Código,Descripción,LOTE,Cantidad,Precio,DTO,Importe
0,04625,RACORD MANGUERA 25 MM,,"25,00","6,86",,"171,50"
1,04134,RACOR MANGUERA 45 ALUM.,,"25,00","6,30",,"157,50"
2,04397,RACOR R/ INT 25 MM,,"35,00","6,30",,"220,50"
3,04396,RACORD R/EXT. 25 MM.,,"25,00","6,30",,"157,50"
4,04237,RACORD R/INT 45 MM,,"25,00","8,96",,"224,00"
5,04322,VALVULA LATON C/RACORD 25 MM,,"10,00","21,28",,"212,80"
6,SIN,RACORD,,,,,
7,02501,"EXTINTOR 5 KG. CO2 ""VALLE-UNION""",,"60,00","35,00",,"2.100,00"
8,,BULTO: 1 PALET,,,,,
9,PESO:,950 KG,,,,,


In [498]:
dat

{'Vat Number': 'B38574281',
 'Total': '10.00',
 'Date': '03/07/2020',
 'Invoice_Number': 'F1-1-01353'}

In [499]:
json_f = {}
json_f['file'] = Ima
json_f['labels'] = dat
json_f['tables'] = df_t
print(json_f)

{'file': 'Makro Invoices/Makro Invoices/Converted/03.07.20_Extintores Valle Union Sl_3243,8_page1.png', 'labels': {'Vat Number': 'B38574281', 'Total': '10.00', 'Date': '03/07/2020', 'Invoice_Number': 'F1-1-01353'}, 'tables':   Código                       Descripción LOTE Cantidad Precio DTO   Importe
0  04625             RACORD MANGUERA 25 MM         25,00   6,86        171,50
1  04134           RACOR MANGUERA 45 ALUM.         25,00   6,30        157,50
2  04397                RACOR R/ INT 25 MM         35,00   6,30        220,50
3  04396              RACORD R/EXT. 25 MM.         25,00   6,30        157,50
4  04237                RACORD R/INT 45 MM         25,00   8,96        224,00
5  04322      VALVULA LATON C/RACORD 25 MM         10,00  21,28        212,80
6    SIN                            RACORD                                   
7  02501  EXTINTOR 5 KG. CO2 "VALLE-UNION"         60,00  35,00      2.100,00
8                           BULTO: 1 PALET                               